In [1]:
import sys
sys.path.append('../vocaldetection/')
import sklearn
import utils
import json
import os
import pandas as pd
import numpy as np

In [2]:
# Read features and labels
FEAT_PATH = os.environ["FEAT_PATH"]
PIECES = os.environ["PIECES_JSON"]

In [3]:
music_files = []

with open(PIECES) as json_file:  
    data = json.load(json_file)

    for music in data.keys():
        music_files.append(FEAT_PATH+music)

In [33]:
train_features = []
train_labels = []


for tf in music_files:
    # Load VGGish audio embeddings
    #vggish = pd.read_csv(tf+"_VGGish_PCA.csv",index_col=None, header=None)
    #vggish = vggish.values
    
    # Load MFCCs features
#     mfccs = pd.read_csv(tf+"_mfcc.csv",index_col=None, header=None)
#     mfccs = mfccs.values.T
    mfcc = np.load(tf+"_mfcc.npy")
    print (mfcc.shape)
    
    # Load VocalVariance features
#     vv = pd.read_csv(tf+"_vocalvar.csv",index_col=None, header=None)
#     vv = vv.values.T
    vv = np.load(tf+"_vocalvar.npy")
    print (vv.shape)

    # Read labels for each frame
#    labels = pd.read_csv(tf+"_lbl.csv",index_col=None, header=None)
#    labels = labels.values[0]
    labels = np.load(tf+"_labels.npy")
    print (labels.shape)
    
    feature_vector = []
    for idx in range(len(labels)):
        feature_vector.append(np.concatenate((mfcc[idx], vv[idx]), axis=0))
        
        
    # Store the feature vector and corresponding label in integer format
    for idx in range(len(feature_vector)):
        train_features.append(feature_vector[idx])
        train_labels.append(labels[idx])
    break

(357, 104)
(357, 5)
(357,)


In [34]:
X = np.array(train_features)
y = np.array(train_labels)

In [35]:
X[0]

array([ 1.07115547e+02, -3.47272873e+01,  1.57313881e+01, -7.95158529e+00,
        1.71852474e+01, -1.61360092e+01,  1.72172463e+00,  5.96364737e+00,
       -1.08871949e+00, -7.30616331e-01, -6.06386423e+00,  1.73046386e+00,
       -1.50515115e+00,  1.03846169e+01,  9.42243099e+00,  8.85262012e+00,
        8.56095219e+00,  7.02073193e+00,  5.01579237e+00,  5.14964819e+00,
        3.47757244e+00,  5.02254057e+00,  3.22590995e+00,  3.13083696e+00,
        3.60949039e+00,  3.08055925e+00,  1.30667923e+02,  1.69300842e+01,
        3.14402905e+01,  1.25162954e+01,  2.96958160e+01, -7.04921961e-01,
        1.21851339e+01,  1.27911100e+01,  9.26175213e+00,  4.96795130e+00,
        5.92182815e-01,  1.05797224e+01,  7.20575094e+00,  1.07875473e+02,
       -3.68285027e+01,  1.72950649e+01, -9.97012806e+00,  1.90751572e+01,
       -1.62606220e+01,  2.87016296e+00,  5.67637873e+00, -1.13273811e+00,
       -4.24645036e-01, -6.02352333e+00,  1.47536767e+00, -1.74072909e+00,
        3.49201083e-01, -

In [10]:
# Split dataset into train, validation and test sets

In [36]:
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

In [37]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

In [38]:
from sklearn.model_selection import KFold # import KFold
seed = 7
kf = KFold(n_splits=10, random_state=seed) # Define the split - into 2 folds 
kf.get_n_splits(X) # returns the number of splitting iterations in the cross-validator

10

In [39]:
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

TRAIN: [ 36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251
 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269
 270 271 272 273 274 275 276 277 278 279 280

In [15]:
# Fit models

# Evaluate models

In [22]:
X.shape

(357, 5)

In [17]:
# Create a scale object
scaler = sklearn.preprocessing.StandardScaler()

# Learn the parameters from the training data only
scaler.fit(train_features)

# Apply the learned parameters to the training and test sets:
train_features_scaled = scaler.transform(train_features)

In [18]:
test_features_scaled = scaler.transform(test_features)

NameError: name 'test_features' is not defined

In [ ]:
# save the scaler to disk
filename = 'scaler_features.sav'
joblib.dump(scaler, filename)

In [19]:
# Use scikit-learn to train a model with the training features we've extracted

# Lets use a SVC with folowing C parameters: 
params = [100, 10, 1, 0.1, 0.01, 0.001, 0.0001]

for c in params:
    clf = sklearn.svm.SVC(C=c)

    # Fit (=train) the model
    clf.fit(train_features_scaled, train_labels)
    
    # save the model to disk
    filename = 'finalized_model_SVM_'+str(c)+'_VGGish.sav'
    print (filename)
    joblib.dump(clf, filename)
    
    # Now lets predict the labels of the test data!
    predictions = clf.predict(test_features_scaled)
    
    # We can use sklearn to compute the accuracy score
    accuracy = sklearn.metrics.accuracy_score(test_labels, predictions)
    print(accuracy)

AttributeError: module 'sklearn' has no attribute 'svm'

In [ ]:
# Now lets predict the labels of the test data!
predictions = clf.predict(test_features_scaled)

In [ ]:
# Biased Guess Accuracy
ones = np.ones(len(predictions))
accuracy = sklearn.metrics.accuracy_score(test_labels, ones)
print(accuracy)

In [ ]:
# SVM Accuracy
accuracy = sklearn.metrics.accuracy_score(test_labels, predictions)
print(accuracy)

In [ ]:
# lets compute the show the confusion matrix:
cm = sklearn.metrics.confusion_matrix(test_labels, predictions)
print(cm)

In [ ]:
fig, ax = plt.subplots()
ax.imshow(cm, interpolation='nearest', cmap='gray')
for i, line in enumerate(cm):
    for j, l in enumerate(line):
        ax.text(j, i, l, size=20, color='green')
ax.set_xticks(range(len(cm)))
ax.set_xticklabels(labels)
ax.set_yticks(range(len(cm)))
ax.set_yticklabels(labels)
ax.set_ylabel('True label')
ax.set_xlabel('Predicted label')
plt.show()

### Random Forest Classifier without scaler

In [ ]:
params = [100, 200, 500, 1000]

for estimators in params:
    clf3 = RandomForestClassifier(n_estimators=estimators)

    # Fit (=train) the model
    clf3.fit(train_features, train_labels)
    
    # save the model to disk
    filename = 'finalized_model_RF_'+str(estimators)+'_VGGish.sav'
    print (filename)
    joblib.dump(clf3, filename)
    
    # Now lets predict the labels of the test data!
    predictions = clf3.predict(test_features)
    # We can use sklearn to compute the accuracy score
    accuracy = sklearn.metrics.accuracy_score(test_labels, predictions)
    print(accuracy)

In [ ]:
# Now lets predict the labels of the test data!
predictions = clf3.predict(test_features)

In [ ]:
# We can use sklearn to compute the accuracy score
accuracy = sklearn.metrics.accuracy_score(test_labels, predictions)
print(accuracy)